Лабораторная работа №2 Халепо О.С. Ктбо3-7

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

Собираем значия из массивов

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
df_test.head(10)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_train.isnull().sum()

label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64

In [ ]:
y_train  = df_train['label'].values
X_train = df_train.drop(columns=['label']).values.reshape(-1,28,28,1)/255.0
X_test = df_test.values.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
y_train_encoded = to_categorical(y_train, num_classes=10)

In [ ]:
model = models.Sequential([

    layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)),
    layers.MaxPool2D(pool_size=2, padding='same'),

    layers.Flatten(),

    layers.Dense(units=256, activation='relu'),
    layers.Dense(units=10, activation='softmax'),
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 256)               3211520   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 3214730 (12.26 MB)
Trainable params: 3214730 (12.26 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.3, verbose=1,
                              patience=2, min_lr=0.00000001)

history = model.fit(
    X_train, y_train_encoded,
    epochs=5,
    validation_split=0.1,
    callbacks=[reduce_lr],
)

Epoch 1/5
1182/1182 [==============================] - 94s 78ms/step - loss: 0.1488 - accuracy: 0.9550 - val_loss: 0.0678 - val_accuracy: 0.9788 - lr: 0.0010
Epoch 2/5
1182/1182 [==============================] - 89s 75ms/step - loss: 0.0495 - accuracy: 0.9838 - val_loss: 0.0567 - val_accuracy: 0.9824 - lr: 0.0010
Epoch 3/5
1182/1182 [==============================] - 87s 73ms/step - loss: 0.0277 - accuracy: 0.9909 - val_loss: 0.0465 - val_accuracy: 0.9855 - lr: 0.0010
Epoch 4/5
1182/1182 [==============================] - 87s 73ms/step - loss: 0.0173 - accuracy: 0.9944 - val_loss: 0.0512 - val_accuracy: 0.9848 - lr: 0.0010
Epoch 5/5
1182/1182 [==============================] - 90s 77ms/step - loss: 0.0122 - accuracy: 0.9960 - val_loss: 0.0477 - val_accuracy: 0.9862 - lr: 0.0010


In [ ]:
# Делаем прогнозы для участия в отборе
predictions = model.predict(X_test)
predictions

875/875 [==============================] - 13s 14ms/step


array([[1.2682868e-11, 2.3305085e-11, 9.9999982e-01, ..., 7.2333904e-08,
        1.2984606e-10, 3.5749836e-10],
       [9.9998301e-01, 2.6733452e-10, 1.1227235e-07, ..., 3.5818124e-07,
        1.0809670e-06, 8.1179685e-08],
       [1.2287641e-12, 4.9514998e-10, 4.7871680e-08, ..., 3.2064765e-08,
        3.0373209e-07, 9.9999458e-01],
       ...,
       [1.7013279e-17, 3.4513391e-14, 2.5388800e-14, ..., 9.4975815e-12,
        8.8672569e-11, 8.9826195e-11],
       [1.5657368e-09, 7.8414456e-13, 6.8820193e-11, ..., 1.6795253e-08,
        8.7984180e-09, 9.9999684e-01],
       [6.9192807e-13, 8.8209301e-12, 9.9999994e-01, ..., 3.2903296e-09,
        1.3349856e-09, 1.7753686e-09]], dtype=float32)

In [ ]:
# Преобразуем выходные данные вероятности модели в предсказания цифр
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)

# Создаем DataFrame для отправки
submission = pd.DataFrame({
    "ImageId": range(1, len(predicted_labels) + 1),
    "Label": predicted_labels
})

# Сохраняем DataFrame в файл CSV
submission.to_csv('submission.csv', index=False)
print(submission)

[2 0 9 ... 3 9 2]
       ImageId  Label
0            1      2
1            2      0
2            3      9
3            4      0
4            5      3
...        ...    ...
27995    27996      9
27996    27997      7
27997    27998      3
27998    27999      9
27999    28000      2

[28000 rows x 2 columns]
